# Hello again, welcome to notebook 5!

In this notebook we will look at:

* Measuring the distances between sentences using the Word Mover's Distance
* Comparing different distance metrics for sentences.

It is recommended that you complete all exercises that are not marked as optional.

Feel free to be creative and write your own code wherever you want!

## Imports

In [1]:
import numpy as np
import string
import os

from nltk import download
download('stopwords')
from nltk.corpus import stopwords

import gensim.downloader as api
from gensim.models.keyedvectors import Word2VecKeyedVectors

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ollie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


We will need to preprocess our sentences using some of the functions from the ```PreparingTextData``` notebook. Let's copy these here:

In [5]:
stop_words = stopwords.words('english')

def basic_tokenizer(sentence):
    out = sentence
    for x in string.punctuation:
        out = out.replace(x, '')
    out = out.lower().split()
    return out

def remove_stopwords(sentence, stop_words):
    return [word for word in sentence if word not in stop_words]

def preprocess(sentence, stop_words):
    clean_sentence = remove_stopwords(basic_tokenizer(sentence), stop_words)
    return clean_sentence

We will also need the Word2Vec vectors that we downloaded in the ```CosineSimilarity``` notebook. We will normalise the vectors as we did before to speed up any similarity calculations.

In [6]:
model = Word2VecKeyedVectors.load("./models/word2vec.model")
model.init_sims(replace=True)

## Lesson 1: Word Mover's Distance

The Word Mover's Distance (WMD) is a measure of the 'distance' between two text documents (or sentences).

<img src="images/wmd_intuition.png" width="600">

All non-stopwords (**bold**) of both sentences are embedded in a word2vec space. The distance between the two sentences is the minimum cumulative distance that the words in the first sentence must travel to exactly match those in the second.

The image below shows how the WMD is calculated for two different cases:

<img src="images/wmd_calculation.png" width="500">

In the top example, we have two sentences, $D_1$ and $D_2$ that have no non-stopwords in common with a third sentences $D_0$. The arrows represent the flow between two words (this is a technical term but is really just 'which word moves where') and are labelled by their contribution to the overall distance. Notice that $D_1$ is closer to $D_0$ than $D_2$. This is what we'd expect!

On the bottom, we have two sentences with different numbers of words. This results in words from $D_3$ being moved to multiple similar words.

(**Source**: http://proceedings.mlr.press/v37/kusnerb15.pdf)

The maths behind the WMD is pretty complicated, but hopefully this high level description should give you a good idea of what it's measuring! Rather than implementing it ourselves, we will use the ```.wmdistance()``` function in Python's ```gensim``` library.

Let's start by preprocessing the sentences from the example above:

In [8]:
D0 = 'The president greets the press in Chicago'
D1 = 'Obama speaks to the media in Illinois'
D2 = 'The band gave a concert in Japan'
D3 = 'Obama speaks in Illinois'

D0_preprocessed = preprocess(D0, stop_words)
D1_preprocessed = preprocess(D1, stop_words)
D2_preprocessed = preprocess(D2, stop_words)
D3_preprocessed = preprocess(D3, stop_words)

print(D0, D0_preprocessed)
print(D1, D1_preprocessed)
print(D2, D2_preprocessed)
print(D3, D3_preprocessed)

The president greets the press in Chicago ['president', 'greets', 'press', 'chicago']
Obama speaks to the media in Illinois ['obama', 'speaks', 'media', 'illinois']
The band gave a concert in Japan ['band', 'gave', 'concert', 'japan']
Obama speaks in Illinois ['obama', 'speaks', 'illinois']


Now calculating the WMD is really easy! Remember that ```model``` is our Word2Vec model from above, and note that the preprocessed sentences ```D0``` and ```D1``` should be fed into ```.wmdistance()``` rather than the sentences themselves.

In [10]:
for D, D_prep in zip([D1, D2, D3], [D1_preprocessed, D2_preprocessed, D3_preprocessed]):
    wmd = model.wmdistance(D0_preprocessed, D_prep)
    print(D0)
    print(D)
    print(f'WMD: {wmd:.4f}')
    print('*' * 42)

The president greets the press in Chicago
Obama speaks to the media in Illinois
WMD: 1.0175
******************************************
The president greets the press in Chicago
The band gave a concert in Japan
WMD: 1.2700
******************************************
The president greets the press in Chicago
Obama speaks in Illinois
WMD: 1.1221
******************************************


### Exercise 1: WMD

In [ ]:
# Q1.1 - The WMD values calculated for the pairs of sentences here are different to those in the
#      - picture above. Why might that be? different word2vec model

In [13]:
# Q1.2 - What's the WMD between two identical sentences?

first_sentence = "hello there, general kenboi"
sentence = preprocess(first_sentence, stop_words)
wmd = model.wmdistance(sentence, sentence)
print(wmd)
# TODO: Call the WMD on your preprocessed `first_sentence` with itself

0.0


In [15]:
# Q1.3 - What's the maximum (finite) WMD that you can find?
#      - What can you say about the bounds of WMD compared to other distance metrics?

second_sentence = "goodbye darth vader"
sentence2 = preprocess(second_sentence, stop_words)
wmd = model.wmdistance(sentence, sentence2)
print(wmd)
# TODO: Call the WMD on your preprocessed sentences

1.1736532357763267


## Lesson 2: Comparing distance metrics

Let's compare the Word Mover's Distance for various sentences to the Jaccard Distance that we calculated in the first notebook. Remember that in order to calculate the Jaccard Distance, we'll need to come up with a Bag of Words representation for our sentences.

The function below calculates a BoW representation and vocabulary for a list of preprocessed sentences, e.g.

```[['obama', 'speaks', 'media', 'illinois'], ['president', 'greets', 'press', 'chicago']]```.

This is slightly different to the function that we saw in the JaccardDistance notebook which took a list of strings (un-tokenized sentences) as input.

In [16]:
def get_bow_and_vocab(sentences):
    vocab = list(set(sentences[0]).union(*sentences[1:]))
    bow = [[s.count(word) for word in vocab] for s in sentences]
    return bow, vocab

In [17]:
bow, vocab = get_bow_and_vocab([D0_preprocessed, D1_preprocessed])

Now we can use the ```calculate_jaccard_dist``` function from the ```JaccardDistance``` notebook to calculate the Jaccard distance between our two sentences.

In [18]:
def calculate_jaccard_dist(u, v):
    intersection = 0
    union = 0
    for i in range(len(u)):
        
        # The word is present in both vectors
        if u[i] > 0 and v[i] > 0:
            intersection += 1
            
        # The word is present in one of the vectors
        if u[i] > 0 or v[i] > 0:
            union += 1
            
    jaccard_dist = 1 - (intersection / union)
    return jaccard_dist

In [19]:
# Jaccard distance between s1 and s2
jaccard = calculate_jaccard_dist(bow[0], bow[1])
print(D0)
print(D1)
print(f'Jaccard distance: {jaccard:.4f}')

The president greets the press in Chicago
Obama speaks to the media in Illinois
Jaccard distance: 1.0000


### Exercise 2: Comparing distance metrics

In [14]:
# Q2.1 - What do you notice about the Jaccard distance compared to the WMD between sentences D0 and D1?
#      - (HINT: Remember to think about the maximum and minimum values of each metric in your answer.)
# Q2.2 - Which metric would you say is more realistic, and why?
#      - (HINT: How does each metric handle synonyms?)

## Lesson 3: Earth Mover's Distance
So far in this notebook, we've made use of the `gensim` library's `.wmdistance()` to calculate the WMD between sentences. In this lesson, we'll implement the Earth Mover's Distance (EMD) - a generalisation of the WMD - from scratch.

To calculate the EMD, we'll need to do the following:
* Represent each document (or sentence) as a list of word vectors. Call these lists $u$ and $v$, and suppose that $u$ has length $m$ and $v$ has length $n$.
* For each pair of vectors $(u_i, v_j)$ where $u_i$ is a vector from $u$ and $v_j$ is a vector from $v$, calculate the Euclidean distance $d_{i,j} = ||u_i - v_j||$. Let this value be the $(i, j)^{th}$ entry in a distance matrix $D$ (an $m \times n$ matrix).
* Find the set of $\min(m, n)$ pairs of words that minimises the total distance between the two documents.
* Calculate the EMD to be the mean of the distances between pairs.

### Example
Suppose that the words 'shop', 'apple', 'pear' and 'supermarket' have the following vector representations:
* 'shop' = $(1, 2)$
* 'apple' = $(5, 7)$
* 'pear' = $(5, 6)$
* 'supermarket' = $(2, 3)$

Then the sentence 'shop apple' is represented by the list $u = [(1, 2), (5, 7)]$ and the sentence 'pear supermarket' by $v = [(5, 6), (2, 3)]$. The distance matrix $D$ is given by

$
D = \begin{pmatrix}
\sqrt{32} & \sqrt{2}\\
1 & 5\\
\end{pmatrix}
$

since:
* $||(1,2) - (5,6)|| = \sqrt{32}$,
* $||(1,2) - (2,3)|| = \sqrt{2}$,

and so on.

Now we have two possible pairings of vectors:
* $((1,2),(5,6))$ and $((5,7),(2,3))$ with total distance $\sqrt{32} + 5$, or
* $((1,2),(2,3))$ and $((5,7),(5,6))$ with total distance $\sqrt{2} + 1$, or

Since $\sqrt{2} + 1 < \sqrt(32) + 5$, we take the second pairing. Thus our sentences 'shop apple' and 'pear supermarket' have an EMD of $\frac{1 + \sqrt{2}}{2}$ where the pairs are ('shop', 'supermarket') and ('apple', 'pear').

### Exercise 3: EMD
Now that we've seen an example of the EMD in action, let's code it up! The hardest part of the EMD calculation is finding the pairs of vectors that minimise the total distance to move. For sentences of only two words it's easy, but as our sentences grow longer it will become much harder! Thankfully there's a Python module `lapsolver` which will be able to do this calculation for us.

In [26]:
from lapsolver import solve_dense

We'll isolate the distance matrix calculation from the main EMD function to make things clearer. Have a look at the `calculate_dist_matrix` function below and check that you understand what it's doing.

In [16]:
def calculate_dist_matrix(u, v):
    mat = np.zeros(shape=(len(u), len(v)), dtype=np.float32)
    for i in range(len(u)):
        for j in range(len(v)):
            mat[i, j] = np.linalg.norm(u[i] - v[j])
    return mat

In [114]:
# Q3.1 - Complete the `calculate_emd_and_pairs` function below.

In [24]:
def calculate_emd_and_pairs(s1, s2, model, stop_words):    
    # TO DO: preprocess the sentences
    clean_s1 =
    clean_s2 =

    # Apply Word2Vec to the sentences
    s1_vec = [model[w] for w in clean_s1]
    s2_vec = [model[w] for w in clean_s2]
    
    # TO DO: calculate the distances between all the word vectors
    dist_matrix =
    
    # Pair the word vectors together
    assignment = solve_dense(dist_matrix)
    pairs = [(clean_s1[i], clean_s2[j]) for i, j in zip(assignment[0], assignment[1])]
    
    # TO DO: take the mean of the distances between pairs
    # HINT: what happens if you index into `dist_matrix` using `assignment`?
    emd =
    
    return emd, pairs

In [21]:
# Q3.2 - Check that your function works by running the cell below.
#      - Compare the EMD values calculated between (D0, D1), (D0, D2) and (D0, D3) to those calculated
#      - using the `gensim` implementation in Lesson 1.
#      - Are they the same or different? Can you think why this might be?

# Q3.3 - Look at the pairs produced when calculating the EMD between D0 and D3.
#      - Why might our EMD implementation be less accurate than the gensim WMD one in this case?

In [25]:
for D in [D1, D2, D3]:
    emd, pairs = calculate_emd_and_pairs(D0, D, model, stop_words)

    print(f'The EMD between "{D0}" and "{D}" is {emd:.4f}')
    print(f'The pairs are: {pairs}')
    print('*' * 115)

The EMD between "The president greets the press in Chicago" and "Obama speaks to the media in Illinois" is 1.0175
The pairs are: [('president', 'obama'), ('greets', 'speaks'), ('press', 'media'), ('chicago', 'illinois')]
*******************************************************************************************************************
The EMD between "The president greets the press in Chicago" and "The band gave a concert in Japan" is 1.2700
The pairs are: [('president', 'band'), ('greets', 'gave'), ('press', 'concert'), ('chicago', 'japan')]
*******************************************************************************************************************
The EMD between "The president greets the press in Chicago" and "Obama speaks in Illinois" is 1.0590
The pairs are: [('president', 'obama'), ('greets', 'speaks'), ('chicago', 'illinois')]
*******************************************************************************************************************
